## Optimize the Model

In [20]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint
import keras_tuner as kt
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

#  Import and read the charity_data.csv.
import pandas as pd
raw_application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
raw_application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [9]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
# Optimzation - drop 'STATUS'
application_df = raw_application_df.drop(columns = ['EIN', 'NAME', 'STATUS'])

In [15]:
# Store the CLASSIFICATION value counts
applications = application_df['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
application_types_to_replace = applications[applications < 528].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Optimization - Create "Rare" and "Very_Rare" CLASSIFICATION bins
# Store the CLASSIFICATION value counts
classifications = application_df['CLASSIFICATION'].value_counts()

# Choose a cutoff values for "Rare" and "Very_Rare" CLASSIFICATION bins
rare_classifications = classifications[(classifications < 1883) & (classifications >= 104)].index.tolist()
very_rare_classifications = classifications[classifications < 104].index.tolist()

# Replace in dataframe
for cls in rare_classifications:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Rare")
for cls in very_rare_classifications:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Very_Rare")

# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Convert categorical data to numeric with `pd.get_dummies`
numeric_application_df = pd.get_dummies(application_df, columns = application_cat)

numeric_application_df = numeric_application_df.astype(int)

numeric_application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [16]:
# Split our preprocessed data into our features and target arrays
y = numeric_application_df['IS_SUCCESSFUL'].values
X = numeric_application_df.drop(columns = 'IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Optimization - Increase neurons 
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  256
hidden_nodes_layer2 = 64

nn = tf.keras.models.Sequential()

# Add the input layer
nn.add(Input(shape=(number_input_features,)))

# Optimization - use "relu" and "tanh" activation functions
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                     │ (None, 256)                 │          10,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,753 (104.50 KB)

 Trainable params: 26,753 (104.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every five epochs
checkpoint_callback = ModelCheckpoint(
    filepath="Results/AlphabetSoupCharity.weights.h5", 
    save_weights_only=True, 
    save_freq=5 * len(X_train_scaled)
)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7059 - loss: 0.5878
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7233 - loss: 0.5621
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7228 - loss: 0.5604
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7334 - loss: 0.5523
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7319 - loss: 0.5531
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7311 - loss: 0.5540
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7313 - loss: 0.5518
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7315 - loss: 0.5491
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7340 - loss: 0.5486
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7310 - loss: 0.5530
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7348 - loss: 0.5424
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [19]:
# Evaluate the model using the test data
model_accuracy, model_loss = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7314 - loss: 0.5573
Loss: 0.7314285635948181, Accuracy: 0.5573010444641113


## Optimization Attempt Two - More Resources

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  256
hidden_nodes_layer2 = 256
hidden_nodes_layer3 = 256
hidden_nodes_layer4 = 128
hidden_nodes_layer5 = 128
hidden_nodes_layer6 = 128

nn = tf.keras.models.Sequential()

# Add the input layer
nn.add(Input(shape=(number_input_features,)))

# Optimization - use "relu", "sigmoid", "leaky_relu" and "tanh" activation functions
# Optimozation - add more layers
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="leaky_relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="leaky_relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every five epochs
checkpoint_callback = ModelCheckpoint(
    filepath="Results/AlphabetSoupCharity.weights.h5", 
    save_weights_only=True, 
    save_freq=5 * len(X_train_scaled)
)

# Train the model
# Optimization - run 500 epochs
fit_model = nn.fit(X_train_scaled,y_train,epochs=500, callbacks=[checkpoint_callback])

Epoch 1/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7007 - loss: 0.5920
Epoch 2/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7272 - loss: 0.5596
Epoch 3/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7366 - loss: 0.5508
Epoch 4/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7272 - loss: 0.5536
Epoch 5/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7279 - loss: 0.5551
Epoch 6/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7335 - loss: 0.5445
Epoch 7/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7371 - loss: 0.5455
Epoch 8/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7349 - loss: 0.5445
Epoch 9/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7333 - loss: 0.5445
Epoch 10/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7330 - loss: 0.5451
Epoch 11/500
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7323 - loss: 0.5469
Epoch 12/500
804/804 ━━━━━━━━━━━━━━━━━━━━

In [8]:
# Evaluate the model using the test data
model_accuracy, model_loss = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7272 - loss: 0.5736
Loss: 0.7272303104400635, Accuracy: 0.573570966720581


## Optimization Attempt Three - Auto-Optimize with Keras Tuner

In [24]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn = tf.keras.models.Sequential()

    # Add the input layer
    nn.add(tf.keras.layers.Input(shape=(number_input_features,)))

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=60,
        step=5), activation=activation))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn

In [25]:
# Set up Hyperband tunner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2
)

# Run the kerastuner search for best hyperparameters
tuner.search(
    X_train_scaled, 
    y_train, 
    epochs=20, 
    validation_data=(X_test_scaled,y_test)
)

Trial 508 Complete [00h 03m 21s]
val_accuracy: 0.7328279614448547

Best val_accuracy So Far: 0.7346938848495483
Total elapsed time: 03h 16m 34s


In [28]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 46, 'num_layers': 4, 'units_0': 3, 'units_1': 5, 'units_2': 1, 'units_3': 1, 'units_4': 9, 'tuner/epochs': 100, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'sigmoid', 'first_units': 41, 'num_layers': 1, 'units_0': 1, 'units_1': 3, 'units_2': 1, 'units_3': 3, 'units_4': 3, 'tuner/epochs': 100, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'sigmoid', 'first_units': 46, 'num_layers': 4, 'units_0': 9, 'units_1': 5, 'units_2': 5, 'units_3': 7, 'units_4': 7, 'tuner/epochs': 12, 'tuner/initial_epoch': 4, 'tuner/bracket': 3, 'tuner/round': 1, 'tuner/trial_id': '0170'}


In [29]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

C:\Users\ecros\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
C:\Users\ecros\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


268/268 - 1s - 2ms/step - accuracy: 0.7347 - loss: 0.5616
Loss: 0.561607301235199, Accuracy: 0.7346938848495483
268/268 - 0s - 2ms/step - accuracy: 0.7342 - loss: 0.5583
Loss: 0.5582844018936157, Accuracy: 0.7342274188995361
268/268 - 1s - 2ms/step - accuracy: 0.7341 - loss: 0.5749
Loss: 0.574857771396637, Accuracy: 0.7341107726097107


In [31]:
# Get the best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 46,
 'num_layers': 4,
 'units_0': 3,
 'units_1': 5,
 'units_2': 1,
 'units_3': 1,
 'units_4': 9,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [32]:
# Compare the performance to the best model
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

C:\Users\ecros\anaconda3\envs\dev\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


268/268 - 1s - 2ms/step - accuracy: 0.7347 - loss: 0.5616
Loss: 0.561607301235199, Accuracy: 0.7346938848495483


In [33]:
# Export our model to HDF5 file
nn.save('Results/AlphabetSoupCharity_Optimization.h5')

In [34]:
# Save the model in the native Keras format
nn.save('Results/AlphabetSoupCharity_Optimization.keras')